In [1]:
import pandas as pd
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\bhupe\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
!pip install fuzzywuzzy

In [2]:
df=pd.read_csv('deduplication.csv')
#reading data

In [3]:
#to get full name and getting correct first name
df["name"]=df.fn+" "+df.ln

In [4]:
#for converting name into title form
#title form will make sure that A is matched with A of Anna only not with that of a Anna
for a in df.name:
    a=a.lower()
    a=a.title()
    

In [5]:
for i in df.index:
    df.name[i]=df.name[i].lower().title()

In [6]:
#making 1968 and 68 to 68, 01 to 1 and made / to -
for i in df.index:
    a=df.dob[i].split('/')
    a[2]=a[2][len(a[2])-2:len(a[2])] #getting the last two digits of the year i.e. 68 from 1968 as well as 68
    a[0]=int(a[0])
    a[1]=int(a[1])
    a[2]=int(a[2])
    df.dob[i]=str(a[0])+'-'+str(a[1])+'-'+str(a[2])

In [7]:
for i in df.index:
    df.fn[i]=df.name[i].split(' ')[0]
    df.ln[i]=df.name[i].replace(df.fn[i]+' ','') #getting last name and deleting the extra space in front of ln

In [8]:
df.drop('name',axis=1,inplace= True)# no need of full name now

In [9]:
original = pd.DataFrame(np.full(df.shape[0],0,int),columns=['original'])


In [12]:
dupli = pd.DataFrame(np.full(df.shape[0],0,int),columns=['dupli'])
# adding a column to dataset for labeling as duplicate or real row
df =pd.concat([df,dupli,original],axis=1)

In [14]:
df  # original data

,ln,dob,gn,fn,dupli,original
0,Smith Jr,1-3-68,F,William,0,0
1,Rothmeyer Jr,1-3-68,F,William,0,0
2,Asby Jr,1-3-68,F,William,0,0
3,Salter Jr,1-3-68,F,William,0,0
4,Salter Jr,1-3-68,F,William,0,0
5,Bland Jr,21-2-62,F,William,0,0
6,Bland Jr,21-2-62,F,William,0,0
7,Bland Jr,21-2-62,F,William,0,0
8,Shaffer Jr,21-2-62,F,William,0,0
9,Shaffer Jr,21-2-62,F,William,0,0


In [19]:
# algorithm for finding duplicates
for a in df.dob.unique():
    dates = df[df.dob==a]
    
    for b in dates.gn.unique():
        sex = dates[dates.gn==b]
        
        for c in dates.fn.unique():
            name = sex[sex.fn==c]
            masters = name.ln.unique().tolist()
            masters.sort(key=len,reverse=True)
            
            for master in masters:
                original_found=False
                for f in name.index:
                    if df.dupli[f] == 0 and df.original[f] == 0:   
                        word_matches = 0 
                        master_pointer = 0
                        slave_pointer = 0
                        
                        slave = name.ln[f]
                        slave_words = slave.split(' ')
                        master_words = master.split(' ')
                        
                        while (slave_pointer < len(slave_words) and master_pointer < len(master_words)):
                            pr = fuzz.partial_ratio(slave_words[slave_pointer], master_words[master_pointer])
                            if pr == 100:
                                word_matches += 1
                                slave_pointer += 1
                                master_pointer += 1
                            else:
                                master_pointer += 1                    
                        
                        if slave == master and not original_found:
                            df.original[f] = 1
                            original_found = True
                        elif word_matches == len(slave_words):
                            df.dupli[f] = 1

C:\Users\bhupe\Anaconda3\lib\site-packages\ipykernel\__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\bhupe\Anaconda3\lib\site-packages\ipykernel\__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
df2 = df[df.original==1]
df2  # filtering out the originals

,ln,dob,gn,fn,dupli,original
0,Smith Jr,1-3-68,F,William,0,1
1,Rothmeyer Jr,1-3-68,F,William,0,1
2,Asby Jr,1-3-68,F,William,0,1
3,Salter Jr,1-3-68,F,William,0,1
5,Bland Jr,21-2-62,F,William,0,1
8,Shaffer Jr,21-2-62,F,William,0,1
10,Bland Iii,21-2-62,F,William,0,1
11,Bland Jr,21-2-62,F,Bill,0,1
14,Bland Jr,8-6-54,F,William,0,1
16,Bland Jr,25-10-53,F,William,0,1


In [ ]:
writer = pd.ExcelWriter('inno.xlsx')
df.to_excel(writer,'Deduplicated')
writer.save()